In [1]:
feature_dict = {i:label for i,label in zip(
                range(4),
                  ('sepal length in cm',
                  'sepal width in cm',
                  'petal length in cm',
                  'petal width in cm', ))}

In [2]:
import pandas as pd
import numpy as np

df = pd.io.parsers.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',',
    )
df.columns = [l for i,l in sorted(feature_dict.items())] + ['class label']
df.dropna(how="all", inplace=True) # to drop the empty line at file-end

df.tail()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class label
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [3]:
from sklearn.preprocessing import LabelEncoder

X = df[['sepal length in cm',
                  'sepal width in cm',
                  'petal length in cm',
                  'petal width in cm',]].values
y = df['class label'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'Setosa', 2: 'Versicolor', 3:'Virginica'}

In [4]:
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))

Mean Vector class 1: [ 5.006  3.418  1.464  0.244]

Mean Vector class 2: [ 5.936  2.77   4.26   1.326]

Mean Vector class 3: [ 6.588  2.974  5.552  2.026]



In [5]:
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((4,4))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1) # make column vector
    overall_mean = overall_mean.reshape(4,1) # make column vector
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

between-class Scatter Matrix:
 [[  63.2121  -19.534   165.1647   71.3631]
 [ -19.534    10.9776  -56.0552  -22.4924]
 [ 165.1647  -56.0552  436.6437  186.9081]
 [  71.3631  -22.4924  186.9081   80.6041]]


In [6]:
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4), mean_vectors):
    class_sc_mat = np.zeros((4,4))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(4,1), mv.reshape(4,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat                             # sum class scatter matrices
print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[ 38.9562  13.683   24.614    5.6556]
 [ 13.683   17.035    8.12     4.9132]
 [ 24.614    8.12    27.22     6.2536]
 [  5.6556   4.9132   6.2536   6.1756]]


In [8]:
S_W_inv = np.linalg.inv(S_W)
S_W_inv

array([[ 0.074 , -0.0371, -0.0613,  0.0238],
       [-0.0371,  0.0976,  0.0188, -0.0628],
       [-0.0613,  0.0188,  0.1005, -0.0606],
       [ 0.0238, -0.0628, -0.0606,  0.2514]])

In [10]:
max_ratio = np.dot(S_W_inv,S_B)
max_ratio

array([[ -3.0247,   1.0485,  -8.0187,  -3.4249],
       [ -5.6209,   2.1525, -15.1084,  -6.3827],
       [  8.029 ,  -2.8654,  21.3687,   9.0984],
       [ 10.6672,  -3.4133,  27.9905,  12.0531]])

In [11]:
# eigenvectors and eigenvalues for the within-class Scatter Matrix:
eig_val_max_ratio, eig_vec_max_ratio = np.linalg.eig(max_ratio)
print( "eig_val_max_ratio=" , eig_val_max_ratio , "\neig_vec_max_ratio=" , eig_vec_max_ratio)

eig_val_max_ratio= [  3.2272e+01   2.7757e-01   3.4225e-15   1.1483e-14] 
eig_vec_max_ratio= [[-0.2049 -0.009  -0.8844 -0.2234]
 [-0.3871 -0.589   0.2854 -0.2523]
 [ 0.5465  0.2543  0.258  -0.326 ]
 [ 0.7138 -0.767   0.2643  0.8833]]


In [13]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_val_max_ratio[i]), eig_vec_max_ratio[:,i]) for i in range(len(eig_val_max_ratio))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
for i in eig_pairs:
    print(i[0])

32.2719577997
0.27756686384
1.14833622793e-14
3.42245892085e-15


In [15]:
eig_pairs

[(32.271957799729812, array([-0.2049, -0.3871,  0.5465,  0.7138])),
 (0.27756686384003953, array([-0.009 , -0.589 ,  0.2543, -0.767 ])),
 (1.1483362279322388e-14, array([-0.2234, -0.2523, -0.326 ,  0.8833])),
 (3.4224589208497691e-15, array([-0.8844,  0.2854,  0.258 ,  0.2643]))]

In [14]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
print('Matrix W:\n', matrix_w)

Matrix W:
 [[-0.2049 -0.009 ]
 [-0.3871 -0.589 ]
 [ 0.5465  0.2543]
 [ 0.7138 -0.767 ]]


In [16]:
transformed = X.dot(matrix_w)
transformed.shape

(150, 2)

In [17]:
transformed

array([[-1.4922, -1.9047],
       [-1.2577, -1.6084],
       [-1.3488, -1.7498],
       [-1.1802, -1.6392],
       [-1.5104, -1.9627],
       [-1.4018, -2.2201],
       [-1.2797, -1.918 ],
       [-1.3784, -1.8195],
       [-1.1165, -1.545 ],
       [-1.3131, -1.5652],
       [-1.5765, -1.9998],
       [-1.2827, -1.7923],
       [-1.3085, -1.5308],
       [-1.37  , -1.6026],
       [-1.9385, -2.2564],
       [-1.7662, -2.5682],
       [-1.6204, -2.3218],
       [-1.4208, -1.9814],
       [-1.496 , -2.0872],
       [-1.4823, -2.1327],
       [-1.351 , -1.7722],
       [-1.3722, -2.1505],
       [-1.6471, -2.0608],
       [-1.0367, -1.9407],
       [-1.1188, -1.716 ],
       [-1.1689, -1.5585],
       [-1.181 , -1.9475],
       [-1.4581, -1.8802],
       [-1.474 , -1.8467],
       [-1.1848, -1.6736],
       [-1.1666, -1.6156],
       [-1.3176, -1.9765],
       [-1.7617, -2.1569],
       [-1.8452, -2.3206],
       [-1.3131, -1.5652],
       [-1.4649, -1.778 ],
       [-1.6288, -1.9337],
 